# Fortnite Players
Find the best players in Fortnite and some of their stats.

## 0.- Imports and Environment

In [1]:
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import json
import requests

## 1.- TEST API
Find an API and create a conection
https://console.fortniteapi.com/

My API credentials

In [2]:
payload = {}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0',
    'Authorization': 'c375dcb72f3ee95b4fc5cc86ba909475',
    'Overwrite-Version': 'v1' #values v1 or v2
}

Test API and results of the API retrieving a user stats. 

Let's try with [NINJA](https://es.wikipedia.org/wiki/Ninja_(streamer)) stats

In [3]:
user_id = '4735ce9132924caf8a5b17789b40f79c' #Ninja Fortnite ID
url = ('https://fortnite-api.theapinetwork.com/prod09/users/public/br_stats_v2?user_id=%s' % (user_id))

In [4]:
response = requests.request('GET', url, headers = headers, data = payload, allow_redirects=False)
results = response.json()

In [5]:
results #json results for Ninja ID

{'data': {'accountId': '4735ce9132924caf8a5b17789b40f79c',
  'username': 'Ninja',
  'stats': {'keyboardmouse': [{'id': 'comp',
     'friendlyName': 'test',
     'entries': [{'mode': 'solo',
       'stats': {'placetop1': 21,
        'kills': 278,
        'matchesplayed': 51,
        'minutesplayed': 0,
        'playersoutlived': 0,
        'score': 0,
        'lastmodified': 0,
        'placetop10': 38,
        'placetop25': 43}},
      {'mode': 'meta',
       'stats': {'placetop1': 12,
        'kills': 162,
        'matchesplayed': 28,
        'minutesplayed': 0,
        'playersoutlived': 0,
        'score': 0,
        'lastmodified': 0,
        'placetop10': 0,
        'placetop25': 0}}]},
    {'id': 'defaultduo',
     'friendlyName': 'test',
     'entries': [{'mode': 'duo',
       'stats': {'placetop1': 1836,
        'kills': 31342,
        'matchesplayed': 4735,
        'minutesplayed': 5475,
        'playersoutlived': 35271,
        'score': 213254,
        'lastmodified': 1562534

In [6]:
data = json_normalize(results)

I found that the stats are separated by type of gamming pad: gamepad, keyboard and touchscreen

In [7]:
data.head()

,data.accountId,data.stats.gamepad,data.stats.keyboardmouse,data.stats.touch,data.username
0,4735ce9132924caf8a5b17789b40f79c,"[{'id': 'defaultduo', 'friendlyName': 'test', ...","[{'id': 'comp', 'friendlyName': 'test', 'entri...","[{'id': 'playground', 'friendlyName': 'test', ...",Ninja


In [8]:
data_keyboard = json_normalize(data['data.stats.keyboardmouse'][0])

In [9]:
data_keyboard.head()

,entries,friendlyName,id
0,"[{'mode': 'solo', 'stats': {'placetop1': 21, '...",test,comp
1,"[{'mode': 'duo', 'stats': {'placetop1': 1836, ...",test,defaultduo
2,"[{'mode': 'solo', 'stats': {'placetop1': 1, 'k...",test,showdowntournament
3,"[{'mode': 'squad', 'stats': {'placetop1': 12, ...",test,classic
4,"[{'mode': 'trios', 'stats': {'placetop1': 10, ...",test,trios


In [10]:
dk_classic = json_normalize(data_keyboard['entries'][0])

In [11]:
dk_classic.head()

,mode,stats.kills,stats.lastmodified,stats.matchesplayed,stats.minutesplayed,stats.placetop1,stats.placetop10,stats.placetop25,stats.playersoutlived,stats.score
0,solo,278,0,51,0,21,38,43,0,0
1,meta,162,0,28,0,12,0,0,0,0


The documentation says that the different pads are:
CONTROLLER = PS4 + Controller, XBOX + Controller, Switch + Controller or PC + Controller.
TOUCHSCREEN = iOS, Android or Switch (WITHOUT CONTROLLER)
KEYBOARD&MOUSE = PC with KB, PS4 with KB or XBOX with KB.

OR the parameter can be passed too with "&platform="
for  PC, PS4, XBOX, Android & iOS and Nintendo Switch stats.


#### How to find a user ID from the username

In [12]:
url = 'https://fortnite-api.theapinetwork.com/users/id?username=MrDoom-'
response = requests.request('GET', url, headers = headers, data = payload)
results = response.json()

In [13]:
results

{'success': False,
 'error': 'Unkown user',
 'eCode': 'unkown.user',
 'errorMessage': 'unknown_epic_user'}

In [14]:
user_id = '53229fd845e54c4db8dc238c8aa3a922' #Test UID retrived by the username
url = ('https://fortnite-api.theapinetwork.com/prod09/users/public/br_stats_v2?user_id=%s' % (user_id))
response = requests.request('GET', url, headers = headers, data = payload, allow_redirects=False)
data = json_normalize(results)
data.head()

,eCode,error,errorMessage,success
0,unkown.user,Unkown user,unknown_epic_user,False


In [15]:
## 2.- TEST WEB
